In [14]:
import random
random.seed(42)

In [15]:

def generate_bushes(level_width):
    bushes = []
    num_bushes = random.randint(1, 60//3)
    print(num_bushes)
    last_x = -1
    for i in range(num_bushes):
        if(last_x == 59):
            last_x -= 1
        x = random.randint(last_x+1, 60-1)
        y = random.randint(10, 12)
        bushes.append([x, y])
        last_x = x
        random.seed()
    return bushes

def generate_ground(level_width):
    # random.seed(1234)  # set random seed for reproducibility
    num_grounds = random.randint(1, 5)  # randomly decide number of grounds
    print(num_grounds)
    grounds = []  # initialize an empty list to store ground coordinates
    prev_x = -1  # initialize previous x coordinate
    for i in range(num_grounds):
        num_pairs = random.randint(1, 7)  # randomly decide number of xy pairs
        pairs = []  # initialize an empty list to store xy pairs
        for j in range(num_pairs):
            x = random.randint(prev_x+1, min(prev_x+4, level_width-1))  # randomly select x value
            y = 7 if j == 0 else pairs[j-1][1] + random.choice([0, 1, 2])  # randomly select y value
            pairs.append((x, y))  # add the xy pair to the list
            prev_x = x  # update the previous x coordinate
            if prev_x+1 >= level_width-1:
                break
        grounds.extend(pairs)  # add the xy pairs to the grounds list
        random.seed()
    return grounds

def generate_sky(level_width):
    # random.seed(1234)  # set random seed for reproducibility
    num_skies = random.randint(1, 5)  # randomly decide number of grounds
    print(num_skies)
    sky = []  # initialize an empty list to store ground coordinates
    prev_x = -1  # initialize previous x coordinate
    for i in range(num_skies):
        num_pairs = random.randint(1, 7)  # randomly decide number of xy pairs
        pairs = []  # initialize an empty list to store xy pairs
        for j in range(num_pairs):
            x = random.randint(prev_x+1, min(prev_x+4, level_width-1))  # randomly select x value
            y = 7 if j == 0 else pairs[j-1][1] + random.choice([0, 1, 2])  # randomly select y value
            pairs.append((x, y))  # add the xy pair to the list
            prev_x = x  # update the previous x coordinate
            if prev_x+1 >= level_width-1:
                break
        sky.extend(pairs)  # add the xy pairs to the grounds list
        random.seed()
    return sky


In [16]:
# Define the level parameters
level_width = 60
level_height = 16
random.seed(1234)

# Define the objects
objects = {
    "bush": [],
    "sky": [],
    "cloud": [],
    "pipe": [],
    "ground": []
}

# Generate the sky objects
sky_width = level_width
sky_height = level_height // 3
objects["sky"] = (generate_sky(level_width))


# Generate the ground object
ground_width = level_width
ground_height = level_height // 3
objects["ground"] = (generate_ground(level_width))

# Generate the pipe objects
pipe_height = 4
pipe_width = 2
pipe_spacing = 5
for x in range(0, level_width - pipe_spacing, pipe_spacing):
    pipe_x = x + random.randint(1, pipe_spacing - pipe_width - 1)
    pipe_y = random.randint(9,12)
    objects["pipe"].append([pipe_x, pipe_y, 4])

# Generate the bush objects

objects["bush"] = generate_bushes(level_width)

# Generate the cloud objects
cloud_width = 3
cloud_height = 1
cloud_spacing = 8
for x in range(0, level_width - cloud_spacing, cloud_spacing):
    cloud_x = x + random.randint(0, cloud_spacing - cloud_width)
    cloud_y = random.randint(0, sky_height - cloud_height)
    objects["cloud"].append([cloud_x, cloud_y])

4
3
5


In [17]:
entities = {
    "CoinBox": [],
    "coinBrick": [],
    "coin": [],
    "Goomba": [],
    "Koopa": [],
    "RandomBox": []
}

# Generate random population of entities
def generate_population(num_entities):
    for i in range(num_entities):
        entity_type = random.choice(list(entities.keys()))
        entities[entity_type].append((random.randint(0, 60), random.randint(7, 16)))
        
# Evaluate fitness of population
def evaluate_fitness():
    fitness = 0
    for entity_type in entities.keys():
        num_entities = len(entities[entity_type])
        if num_entities == 0:
            fitness -= 10  # penalty for having no entities of this type
        elif num_entities == 1:
            fitness += 10  # bonus for having exactly one entity of this type
        else:
            fitness += 5 * num_entities  # bonus for having multiple entities of this type
    return fitness

# Run genetic algorithm
def genetic_algorithm(num_entities, num_generations):
    # Generate initial population
    generate_population(num_entities)
    
    # Evaluate fitness of initial population
    best_fitness = evaluate_fitness()
    best_entities = entities.copy()
    
    # Run generations
    for i in range(num_generations):
        # Generate new population
        new_entities = {entity_type: [] for entity_type in entities.keys()}
        for entity_type in entities.keys():
            num_entities = len(entities[entity_type])
            for j in range(num_entities):
                # Mutate existing entity
                if random.random() < 0.5:
                    new_x = entities[entity_type][j][0] + random.randint(-10, 10)
                    new_y = entities[entity_type][j][1] + random.randint(-10, 10)
                    new_entities[entity_type].append((new_x, new_y))
                # Crossover two existing entities
                else:
                    other_entity_type = random.choice(list(entities.keys()))
                    other_num_entities = len(entities[other_entity_type])
                    if other_num_entities > 0:
                        other_entity_index = random.randint(0, other_num_entities-1)
                        new_x = (entities[entity_type][j][0] + entities[other_entity_type][other_entity_index][0]) // 2
                        new_y = (entities[entity_type][j][1] + entities[other_entity_type][other_entity_index][1]) // 2
                        new_entities[entity_type].append((new_x, new_y))
        
        # Update current population with new population
        entities.update(new_entities)
        
        # Evaluate fitness of new population
        fitness = evaluate_fitness()
        
        # Update best fitness and best entities if necessary
        if fitness > best_fitness:
            best_fitness = fitness
            best_entities = entities.copy()
    
    # Return best entities
    return best_entities


In [18]:
# Create the level
level = {
    "id": 0,
    "length": level_width,
    "level": {
        "objects": objects,
        "layers": {
            "sky": {"x": [0, level_width], "y": [0, 13]},
            "ground": {"x": [0, level_width], "y": [14, 16]}
        },
        "entities": genetic_algorithm(len(entities), 5)
    }
}


In [19]:
import json
level_json = json.dumps(level)
level_json

'{"id": 0, "length": 60, "level": {"objects": {"bush": [[56, 10], [59, 12], [59, 11], [59, 10], [59, 10]], "sky": [[0, 7], [2, 7], [4, 9], [5, 9], [6, 9], [10, 10], [13, 11], [17, 11], [20, 7], [22, 9], [23, 10], [27, 11], [29, 7], [31, 8], [32, 10], [35, 12], [37, 12], [40, 13], [43, 13]], "cloud": [[5, 1], [12, 0], [20, 4], [26, 3], [35, 1], [43, 1], [51, 1]], "pipe": [[2, 11, 4], [7, 9, 4], [11, 9, 4], [17, 12, 4], [21, 9, 4], [27, 10, 4], [31, 9, 4], [36, 11, 4], [42, 11, 4], [47, 11, 4], [52, 10, 4]], "ground": [[0, 7], [2, 8], [4, 8], [5, 7], [6, 8], [9, 7], [13, 8], [16, 10]]}, "layers": {"sky": {"x": [0, 60], "y": [0, 13]}, "ground": {"x": [0, 60], "y": [14, 16]}}, "entities": {"CoinBox": [[50, 16], [8, 7]], "coinBrick": [[9, 2]], "coin": [[51, 0]], "Goomba": [[20, 12], [35, 2]], "Koopa": [], "RandomBox": []}}}'